In [ ]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from google.colab import drive

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

import warnings
warnings.filterwarnings("ignore") # `do not disturbe` mode

In [ ]:

drive.mount('/Drive_content')

In [ ]:
TRAIN_PATH = "/Drive_content/MyDrive/Colab Notebooks/JPX_data/train_files/"
FINANCIALS = "financials.csv"
OPTIONS = "options.csv"
STOCK_PRICES = "stock_prices.csv"
SEC_STOCK_PRICES = "secondary_stock_prices.csv"
TRADES = "trades.csv"

In [ ]:
# file_name = TRAIN_PATH + FINANCIALS
df_financials = pd.read_csv(TRAIN_PATH + FINANCIALS, parse_dates=["Date"])
df_options = pd.read_csv(TRAIN_PATH + OPTIONS)
df_stock_prices = pd.read_csv(TRAIN_PATH + STOCK_PRICES, parse_dates=["Date"])
df_sec_stock_prices = pd.read_csv(TRAIN_PATH + SEC_STOCK_PRICES)
df_trades = pd.read_csv(TRAIN_PATH + TRADES)

In [ ]:
df_financials.head()

In [ ]:
df_financials.info()

In [ ]:
df_options.head()

In [ ]:
df_stock_prices.head()

In [ ]:
df_stock_prices.info()

In [ ]:
for col in df_stock_prices.columns:
  missing_values = df_stock_prices[col].isnull().sum()
  print(f"In column {col} {missing_values} values is missing")

In [ ]:
df_stock_prices["Date"][df_stock_prices["Open"].isnull()].value_counts()

1988 stocks not traded 2020-10-01
Всего нул значений по столбцу "Open" 7608

In [ ]:
df_stock_prices["SecuritiesCode"].nunique()

In [ ]:
df_stock_prices["Volume"][df_stock_prices["Open"].isnull()].sum()

Бумаги не торговались в эти даты.
Не буду ни на что заменять пропущенные значения, просто дропну их / 
Оказалось, это плохая идея, заменю пропуски предыдущими значениями по каждой бумаге в отдельности

In [ ]:
#df_stock_prices.dropna(subset=["Open","High","Low","Close"], inplace=True)

In [ ]:
df_stock_prices[["SecuritiesCode", "Open","High","Low","Close"]][df_stock_prices['SecuritiesCode'] == 1301]

In [ ]:
df_stock_prices = df_stock_prices.sort_values(by="Date", ascending=False)

In [ ]:
def stock_prices_prep(data):
    df = pd.DataFrame()
    df_temp = pd.DataFrame()
    for code in data['SecuritiesCode'].unique():
        df_temp = data[data['SecuritiesCode'] == code].sort_values(by="Date",  ascending=False)

        df_temp[["High", "Low", "Close", "Open"]] = df_temp[["High", "Low", "Close", "Open"]].fillna(method="bfill")
        df_temp[["High", "Low", "Close", "Open"]] = df_temp[["High", "Low", "Close", "Open"]].fillna(0)
        
        df = df.append(df_temp)

    data = df.sort_values(by="Date")


    data["Adj_Close"] = data["Close"] * data["AdjustmentFactor"]
    data['ExpectedDividend'].fillna(0, inplace=True)
  # data.drop("AdjustmentFactor", axis=1, inplace=True)

    return data

In [ ]:
df_stock_prices = stock_prices_prep(df_stock_prices)

In [ ]:
df_stock_prices[df_stock_prices['Close'].isna()]

In [ ]:
df_stock_prices["Day_change_pct"] = (df_stock_prices["Close"] / df_stock_prices["Open"] - 1) * 100


In [ ]:
df_stock_prices.head(10)

In [ ]:
df_stock_prices.assign(Is_outlier = 0)

In [ ]:
def find_outliers(data, col):
  Q1 = data[col].quantile(0.25)
  Q3 = data[col].quantile(0.75)
  IQR = Q3 - Q1
  lower = Q1 - 1.5 * IQR
  upper = Q3 + 1.5 * IQR

  print(f"Lower is {lower}, upper is {upper}, IQR is {IQR}")

  data["Is_outlier"] = np.where(data[col] < lower, 1, 0)
  data["Is_outlier"] = np.where(data[col] > upper, 1, 0)
  

  return data


In [ ]:
find_outliers(df_stock_prices, "Day_change_pct")

In [ ]:
df_stock_prices[df_stock_prices["Is_outlier"] == 1]

In [ ]:
# !pip install plotly

In [ ]:
import plotly
import plotly.graph_objs as go

In [ ]:
df_stock_1813 = df_stock_prices[df_stock_prices["SecuritiesCode"] == 1813]
df_stock_1879 = df_stock_prices[df_stock_prices["SecuritiesCode"] == 1879]
df_stock_3540 = df_stock_prices[df_stock_prices["SecuritiesCode"] == 3540]
df_stock_1813.head()

In [ ]:
df_stock_1813.set_index('Date', inplace=True)
df_stock_1879.set_index('Date', inplace=True)
df_stock_3540.set_index('Date', inplace=True)
df_stock_1879.head()

In [ ]:
def stock_by_SecuritiesCode(data, code):
    df = data[data['SecuritiesCode'] == code]
    df.set_index('Date', inplace=True)
    return df

In [ ]:
stock_by_SecuritiesCode(df_stock_prices, 1813)


In [ ]:
from plotly.subplots import make_subplots
# fig = go.Figure() # for combined plot (1 area - 2 lines)
fig = make_subplots(rows=2, cols=1)
fig.add_trace(go.Scatter(x=df_stock_1813.index, 
                         y=df_stock_1813["Day_change_pct"], 
                         name='SecuritiesCode=1813',
                         mode='lines+markers',
                         marker=dict(color=df_stock_1813["Is_outlier"], colorscale='Inferno')),
                         1, 1)
fig.add_trace(go.Scatter(x=df_stock_1879.index, 
                         y=df_stock_1879["Day_change_pct"], 
                         name='SecuritiesCode=1879',
                         mode='lines+markers',
                         marker=dict(color=df_stock_1879["Is_outlier"], colorscale='Inferno')),
                         2, 1)
fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#008000')
fig.update_layout(legend_orientation="h",
                  margin=dict(l=0, r=0, t=30, b=0),
                  title="Stock prices",
                  hovermode="x")
fig.update_traces(hoverinfo="all", hovertemplate="Date: %{x}<br>Day Change: %{y}")
fig.update_yaxes(title="Price Day Change", col=1, row=1)
fig.update_yaxes(title="Price Day Change", col=1, row=2)
fig.update_xaxes(title="Date", col=1, row=2)

fig.show()

**Sklearn Time Series** ⏰


---



In [ ]:
from sklearn.metrics import (mean_absolute_error, mean_squared_error,
                             mean_squared_log_error, median_absolute_error,
                             r2_score)

In [ ]:
import warnings
warnings.filterwarnings("ignore") # `do not disturbe` mode

**Moving average**

In [ ]:
def moving_average(series, n):
  """
  Calculate average of last n observations
  """
  return np.average(series[-n: ])

In [ ]:
moving_average(stock_by_SecuritiesCode(df_stock_prices, 1879)["Adj_Close"], 24)

In [ ]:
def plot_price_volume(data, code):
    df_ploted = stock_by_SecuritiesCode(data, code)
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02, row_heights=[400, 200])
    fig.add_trace(go.Scatter(x=df_ploted.index, 
                         y=df_ploted["Adj_Close"], 
                         name=f'SecuritiesCode={code}',
                         mode='lines+markers',
                         marker=dict(color=df_ploted["Is_outlier"], colorscale='Inferno')),
                         1, 1)
    fig.add_trace(go.Bar(x=df_ploted.index, 
                         y=df_ploted["Volume"], 
                         name=f'SecuritiesCode={code}'),
                         2, 1)
    fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
    fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#008000')
    fig.update_layout(legend_orientation="h",
                  margin=dict(l=0, r=0, t=30, b=0),
                  title="Stock prices",
                  hovermode="x")
    fig.update_traces(hoverinfo="all", hovertemplate="Date: %{x}<br>Adj Adjusted Close Price: %{y}", col=1, row=1)
    fig.update_traces(hoverinfo="all", hovertemplate="Date: %{x}<br>Volume: %{y}", col=1, row=2)
    fig.update_yaxes(title="Adj Close Price", col=1, row=1)
    fig.update_yaxes(title="Volume", col=1, row=2)
    fig.update_xaxes(title="Date", col=1, row=2)

    fig.show()

In [ ]:
plot_price_volume(df_stock_prices, 3540)

In [ ]:
def plotMovingAverage(
    series, window, plot_intervals=False, scale=1.96, plot_anomalies=False
):

    """
        series - pd.Series
        window - rolling window size
        plot_intervals - show confidence intervals
        plot_anomalies - show anomalies

    """
    rolling_mean = series.rolling(window=window).mean()
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=series.index, 
                         y=rolling_mean, 
                         name='Rolling mean trend',
                         mode='lines',
                         line=dict(color='#A4133C', width=3)))

    fig.update_layout(legend_orientation="h",
                  margin=dict(l=0, r=0, t=30, b=0),
                  title=f"Moving average\n window size = {window}"
                  )
    fig.update_traces(hoverinfo="all", hovertemplate="Date: %{x}<br>Moving AVG Adj Close: %{y}")
    fig.update_yaxes(title="Moving AVG Adj Close")
    fig.update_xaxes(title="Date")
    

    # Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bond = rolling_mean - (mae + scale * deviation)
        upper_bond = rolling_mean + (mae + scale * deviation)
        fig.add_trace(go.Scatter(x=series.index, 
                         y=upper_bond, 
                         name='Upper Bond / Lower Bond',
                         mode='lines',
                         line=dict(color='#f29559', dash='dash')))
        fig.add_trace(go.Scatter(x=series.index, 
                         y=lower_bond,
                         name='Upper Bond / Lower Bond', 
                         mode='lines',
                         line=dict(color='#f29559', dash='dash')))

        # Having the intervals, find abnormal values
        if plot_anomalies:
            
            anomalies = pd.Series(data=None, index=series.index)
            anomalies[series < lower_bond] = series[series < lower_bond]
            anomalies[series > upper_bond] = series[series > upper_bond]
            fig.add_trace(go.Scatter(x=anomalies.index, 
                         y=anomalies,
                         mode='markers',
                         name='Anomalies',
                         marker=dict(color='#eb5e28', size=20, line=dict(color='#9a031e', width=2))))
            
    fig.add_trace(go.Scatter(x=series.index[window:], 
                         y=series[window:], 
                         name='Actual values',
                         mode='lines+markers',
                         marker=dict(color='rgba(13, 59, 102, .8)', size=7)
                         ))
    
    # fig.update_xaxes(rangeslider_visible=True)
    fig.update_xaxes(
    rangebreaks=[
        dict(bounds=["sat", "mon"]) #hide weekends
        # dict(values=["2015-12-25", "2016-01-01"])  # hide Christmas and New Year's
    ]
    )
    fig.show()

In [ ]:
df = df_stock_1813
df.info()

В 'Day_change_pct' нет пустых значений, тк мы дропнули выше строки с пустыми значениями в столбцах Open	High	Low	Close / значения не удалены, а заменены

In [ ]:
# plotMovingAverage(df_stock_1813["Adj_Close"], 30, plot_intervals=True, plot_anomalies=True)
# stock_by_SecuritiesCode(df_stock_prices, 1813)
plotMovingAverage(stock_by_SecuritiesCode(df_stock_prices, 1813)["Adj_Close"], 30, plot_intervals=True, plot_anomalies=True)

In [ ]:
plotMovingAverage(stock_by_SecuritiesCode(df_stock_prices, 1813)["Day_change_pct"], 30, plot_intervals=True, plot_anomalies=True)

Посмотрим на разброс значений в "AdjustmentFactor"

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=df_stock_prices["AdjustmentFactor"][df_stock_prices["SecuritiesCode"] == 7638], nbinsx=20))

# fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
# fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#008000')
# fig.update_layout(legend_orientation="h",
#                   margin=dict(l=0, r=0, t=30, b=0),
#                   title="")
# fig.update_yaxes(title=" ")
# fig.update_xaxes(title="Date")

fig.show()

In [ ]:
df_stock_prices[df_stock_prices["AdjustmentFactor"] >= 3].sort_values(by=['AdjustmentFactor'], ascending=False)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(y=df_stock_prices["AdjustmentFactor"]))



fig.show()

Посмотрим, что это за компании и что там происходило с ценой

In [ ]:
new_path = "/Drive_content/MyDrive/Colab Notebooks/JPX_data/"
file_name = "stock_list.csv"
df_stock_list = pd.read_csv(new_path + file_name)
df_stock_list.head()

In [ ]:
df_stock_list[df_stock_list["SecuritiesCode"] == 7638]

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_by_SecuritiesCode(df_stock_prices, 7638).index, 
                         y=stock_by_SecuritiesCode(df_stock_prices, 7638)["Volume"],
                         name='SecuritiesCode=7638',
                         mode='lines+markers'))

fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#008000')
fig.update_layout(legend_orientation="h",
                  margin=dict(l=0, r=0, t=30, b=0),
                  title="Stock prices")
fig.update_yaxes(title=" ")
fig.update_xaxes(title="Date")
fig.show()

Если AdjustmentFactor не равен 1, то домножить на него все более ранние значения цены (а объем разделить). AdjustmentFactor не учитывает дивиденды, поэтому на двд нужна доп корректировка - до экс-дивидендной даты цена закрытия  должна быть домножена на понижающий коэффициент, равный (цена закрытия перед экс-двд датой - ДВД) / цена закрытия перед экс-двд датой.

In [ ]:
stock_by_SecuritiesCode(df_stock_prices, 7638).tail(10)

Корректировка на ДВД на примере этой бумаги. ДВД ожидаемый указан в дате 26 марта 2021. В спецификации сказано, что двд указывается тут за 2 рабочих дня до экс-двд даты. Получаем, что экс-двд дата = 30 марта. По предыдущей цене закрытия и размеру двд рассчитываем корректирующий коэф. В данном случае (1168 - 50) / 1168 = 0,957192 (округлить)

In [ ]:
df_stock_prices = df_stock_prices.assign(Adj = 1)

In [ ]:
df_stock_prices = df_stock_prices.sort_values(by='Date', ascending=False)
df_stock_prices.head()

In [ ]:
def calc_main_adjusting_factor(sorted_data):
    df = pd.DataFrame()
    df_temp = pd.DataFrame()
    for code in sorted_data['SecuritiesCode'].unique():
        df_temp = sorted_data[sorted_data['SecuritiesCode'] == code]

        df_temp['Adj'] = df_temp['AdjustmentFactor'].cumprod()

        df = df.append(df_temp)

    df = df.sort_values(by='Date', ascending=False)

    return df




In [ ]:
df_stock_prices = calc_main_adjusting_factor(df_stock_prices)

Пересчитаем столбец Adj_Close

In [ ]:
df_stock_prices['Adj_Close'] = df_stock_prices['Close'] * df_stock_prices['Adj']

In [ ]:
df_stock_prices = df_stock_prices.assign(Volume_Adj = 0)

In [ ]:
df_stock_prices['Volume_Adj'] = df_stock_prices['Volume'] / df_stock_prices['Adj']



Посмотрим теперь на график Volume_Adj

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=stock_by_SecuritiesCode(df_stock_prices, 7638).index, 
                         y=stock_by_SecuritiesCode(df_stock_prices, 7638)["Volume_Adj"],
                         name='SecuritiesCode=1813',
                         mode='lines+markers'))

fig.update_yaxes(zeroline=True, zerolinewidth=2, zerolinecolor='LightPink')
fig.update_xaxes(zeroline=True, zerolinewidth=2, zerolinecolor='#008000')
fig.update_layout(legend_orientation="h",
                  margin=dict(l=0, r=0, t=30, b=0),
                  title="Stock prices")
fig.update_yaxes(title=" ")
fig.update_xaxes(title="Date")
fig.show()

In [ ]:
df_stock_1813 = df_stock_prices[df_stock_prices["SecuritiesCode"] == 1813]
df_stock_1879 = df_stock_prices[df_stock_prices["SecuritiesCode"] == 1879]
df_stock_7638 = df_stock_prices[df_stock_prices["SecuritiesCode"] == 7638]


In [ ]:
df_stock_1813.set_index('Date', inplace=True)
df_stock_1879.set_index('Date', inplace=True)
df_stock_7638.set_index('Date', inplace=True)

In [ ]:
plotMovingAverage(stock_by_SecuritiesCode(df_stock_prices, 7638)["Adj_Close"], 30, plot_intervals=True, plot_anomalies=True)

In [ ]:
plotMovingAverage(stock_by_SecuritiesCode(df_stock_prices, 1813)["Adj_Close"], 30, plot_intervals=True, plot_anomalies=True)

In [ ]:
plotMovingAverage(stock_by_SecuritiesCode(df_stock_prices, 1879)["Adj_Close"], 30, plot_intervals=True, plot_anomalies=True)

Осталось сделать поправку на дивиденды

In [ ]:

def calc_dvd_adjusting_factor_v1(sorted_data, sec_list, shifted_col):
    sorted_data = sorted_data.assign(Close_shifted=0)
    sorted_data = sorted_data.assign(Adj_dvd=1)
    sorted_data = sorted_data.assign(Adj_dvd_cum=1)
    for code in sorted_data[sec_list].unique():
        sorted_data["Close_shifted"][sorted_data[sec_list] == code] = sorted_data[shifted_col][sorted_data[sec_list] == code].shift(periods=1, fill_value=0)
        sorted_data['Adj_dvd'][sorted_data[sec_list] == code] = (sorted_data["Close_shifted"][sorted_data[sec_list] == code]  - sorted_data['ExpectedDividend'][sorted_data[sec_list] == code]) / sorted_data['Close_shifted'][sorted_data[sec_list] == code]
        sorted_data['Adj_dvd'][sorted_data[sec_list] == code].fillna(1, inplace=True)
        sorted_data['Adj_dvd_cum'][sorted_data[sec_list] == code] = sorted_data['Adj_dvd'][sorted_data[sec_list] == code].cumprod()
        
          # датафрейм отсортирован по дате по убыванию
          # находим ExpectedDividend > 0, запоминаем двд
          # сдвигаемся на одну строку выше и берем цену Close. Рассчитываем коэф-т для корректировки, пишем коэф-т в столбец 'Adj_dvd' в той же строке. / делаем сдвиг столбца 'Close' вниз на 1 
          # 
        
    return sorted_data



In [ ]:
df_stock_prices_copy = df_stock_prices

In [ ]:
# %%time
# calc_dvd_adjusting_factor_v1(df_stock_prices, "SecuritiesCode", "Close")

In [ ]:

def calc_dvd_adjusting_factor_v2(sorted_data, sec_list, shifted_col):
    sorted_data = sorted_data.assign(Close_shifted=0)
    sorted_data = sorted_data.assign(Adj_dvd=1)
    sorted_data = sorted_data.assign(Adj_dvd_cum=1)
    df = pd.DataFrame()
    df_temp = pd.DataFrame()
    for code in sorted_data[sec_list].unique():
        df_temp = sorted_data[sorted_data[sec_list] == code]
        df_temp["Close_shifted"] = df_temp[shifted_col].shift(periods=1, fill_value=0)
        df_temp['Adj_dvd'] = (df_temp["Close_shifted"] - df_temp['ExpectedDividend']) / df_temp['Close_shifted']
        df_temp['Adj_dvd'].fillna(1, inplace=True)
        df_temp['Adj_dvd_cum'] = df_temp['Adj_dvd'].cumprod()

        df = df.append(df_temp)
          # датафрейм отсортирован по дате по убыванию
          # находим ExpectedDividend > 0, запоминаем двд
          # сдвигаемся на одну строку выше и берем цену Close. Рассчитываем коэф-т для корректировки, пишем коэф-т в столбец 'Adj_dvd' в той же строке. / делаем сдвиг столбца 'Close' вниз на 1 
    



    return df



In [ ]:
%%time
df_stock_prices_Adj = calc_dvd_adjusting_factor_v2(df_stock_prices_copy, "SecuritiesCode", "Close")
# df_stock_prices_Adj

calc_dvd_adjusting_factor_v2 работает в 2 раза быстрее, чем calc_dvd_adjusting_factor_v1

In [ ]:
df_stock_prices_Adj['Adj_Close'] = df_stock_prices_Adj['Close'] * df_stock_prices_Adj['Adj'] * df_stock_prices_Adj['Adj_dvd_cum']

In [ ]:
df_stock_prices_Adj = df_stock_prices_Adj.sort_values(by='Date')

In [ ]:
stock_by_SecuritiesCode(df_stock_prices_Adj, 9083).head(40)

**Getting rid of non-stationarity and building SARIMA**


---



In [ ]:
# def tsplot(y, lags=None, figsize=(12, 7), style="bmh"):
#     """
#         Plot time series, its ACF and PACF, calculate Dickey–Fuller test

#         y - timeseries
#         lags - how many lags to include in ACF, PACF calculation
#     """
#     if not isinstance(y, pd.Series):
#         y = pd.Series(y)

#     with plt.style.context(style):
#         fig = plt.figure(figsize=figsize)
#         layout = (2, 2)
#         ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
#         acf_ax = plt.subplot2grid(layout, (1, 0))
#         pacf_ax = plt.subplot2grid(layout, (1, 1))

#         ts_ax.plot(y)
#         p_value = sm.tsa.stattools.adfuller(y)[1]
#         ts_ax.set_title(
#             "Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}".format(p_value)
#         )
#         smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
#         smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
#         plt.tight_layout()